In [ ]:
# Lab 2: RAG 기초 - 소방기본법 검색 (40분) - 실습 버전
# 소방청 민원처리 AI Agent 교육

"""
학습 목표:
- RAG(Retrieval Augmented Generation) 개념 이해
- PDF 텍스트 추출 및 청킹
- 임베딩과 벡터 검색
- ChromaDB 사용법
"""

# ============================================================
# 환경 설정
# ============================================================

import sys, os
if 'google.colab' in sys.modules:
    !pip install openai chromadb pdfplumber -q

import getpass
from openai import OpenAI
import pdfplumber
import chromadb
from chromadb.config import Settings
from typing import List, Dict

# API 키 설정
print("[API 키 입력] OpenAI API 키를 입력하세요")
api_key = getpass.getpass("API Key: ")
client = OpenAI(api_key=api_key)

print("[완료] 환경 설정 완료!\n")

# ============================================================
# PDF 파일 업로드
# ============================================================

from google.colab import files

print("[파일 업로드] 소방기본법.pdf (10페이지)를 업로드하세요")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"[완료] {filename} 업로드 완료")

# ============================================================
# Lab 1 함수 재사용 (이미 구현됨)
# ============================================================

def classify_complaint(text: str) -> Dict[str, str]:
    """Lab 1에서 만든 민원 분류 함수"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """소방 민원을 분류하고 다음 형식으로 답변하세요:

카테고리: [화재신고/소방시설/법규위반/일반문의]
긴급도: [높음/보통/낮음]
키워드: [핵심 키워드 3개]
요약: [한 줄 요약]"""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=200
    )
    
    result_text = response.choices[0].message.content
    
    return {
        "원문": text,
        "분류_결과": result_text
    }

# ============================================================
# 실습 1: PDF 텍스트 추출
# ============================================================

print("\n" + "="*60)
print("[실습 1] PDF 텍스트 추출")
print("="*60)

def extract_text_from_pdf(pdf_path: str, max_pages: int = None) -> List[Dict]:
    """
    PDF에서 텍스트 추출 (페이지별)
    
    TODO: pdfplumber를 사용하여 PDF 텍스트 추출하기
    
    힌트:
    1. pdfplumber.open(pdf_path) 사용
    2. with 문으로 PDF 열기
    3. pdf.pages로 페이지 접근
    4. page.extract_text()로 텍스트 추출
    5. 각 페이지별로 딕셔너리 생성
    
    반환 형식:
    [
        {"page_num": 1, "text": "...", "char_count": 1000},
        {"page_num": 2, "text": "...", "char_count": 950},
        ...
    ]
    """
    pages_data = []
    
    # TODO: pdfplumber로 PDF 열기
    # with pdfplumber.open(pdf_path) as pdf:
    #     total_pages = ...
    #     for i in range(total_pages):
    #         page = pdf.pages[i]
    #         text = page.extract_text()
    #         pages_data.append({...})
    
    return pages_data

# PDF 읽기 테스트
print("\n[테스트]")
fire_law_pages = extract_text_from_pdf("소방기본법.pdf", max_pages=10)

if fire_law_pages:
    print(f"[완료] 총 {len(fire_law_pages)} 페이지 추출")
    print(f"\n[샘플] 1페이지 내용 (처음 300자):")
    print(fire_law_pages[0]['text'][:300])
    print("...")
else:
    print("[아직 구현되지 않았습니다]")

# ============================================================
# 실습 2: 텍스트 청킹 (Chunking)
# ============================================================

print("\n\n" + "="*60)
print("[실습 2] 텍스트 청킹 - 작은 단위로 분할")
print("="*60)

def chunk_text(pages_data: List[Dict], chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    """
    텍스트를 작은 청크로 분할
    
    TODO: 슬라이딩 윈도우 방식으로 텍스트 청킹하기
    
    왜 청킹이 필요한가?
    - 전체 문서는 너무 길어서 검색 정확도가 떨어짐
    - 작은 단위로 나누면 관련 부분만 정확히 찾을 수 있음
    
    힌트:
    1. 각 페이지의 텍스트를 chunk_size만큼 자르기
    2. start 위치를 overlap만큼 겹치게 이동
    3. 각 청크에 고유 ID, 페이지 번호 저장
    
    반환 형식:
    [
        {"id": "law_chunk_0", "text": "...", "page": 1, "start_pos": 0},
        {"id": "law_chunk_1", "text": "...", "page": 1, "start_pos": 450},
        ...
    ]
    """
    chunks = []
    chunk_id = 0
    
    for page_data in pages_data:
        text = page_data['text']
        page_num = page_data['page_num']
        
        # TODO: 슬라이딩 윈도우로 청킹하기
        # start = 0
        # while start < len(text):
        #     end = start + chunk_size
        #     chunk_text = text[start:end]
        #     if chunk_text.strip():
        #         chunks.append({...})
        #         chunk_id += 1
        #     start = end - overlap
    
    return chunks

# 청킹 테스트
if fire_law_pages:
    print("\n[테스트]")
    law_chunks = chunk_text(fire_law_pages, chunk_size=500, overlap=50)
    
    if law_chunks:
        print(f"[완료] 총 {len(law_chunks)}개 청크 생성")
        print(f"\n[샘플] 첫 번째 청크:")
        print(f"ID: {law_chunks[0]['id']}")
        print(f"페이지: {law_chunks[0]['page']}")
        print(f"길이: {len(law_chunks[0]['text'])} 문자")
        print(f"내용: {law_chunks[0]['text'][:200]}...")
    else:
        print("[아직 구현되지 않았습니다]")
else:
    print("[PDF를 먼저 읽어야 합니다]")
    law_chunks = []

# ============================================================
# 실습 3: 임베딩 생성
# ============================================================

print("\n\n" + "="*60)
print("[실습 3] 임베딩 - 텍스트를 숫자 벡터로 변환")
print("="*60)

def get_embedding(text: str) -> List[float]:
    """
    텍스트를 벡터로 변환
    
    TODO: OpenAI Embeddings API 호출하기
    
    임베딩이란?
    - 텍스트를 의미를 담은 숫자 배열로 변환
    - 의미가 비슷한 텍스트는 비슷한 숫자 배열을 가짐
    
    힌트:
    - client.embeddings.create() 사용
    - model: "text-embedding-3-small"
    - input: text
    - 반환: response.data[0].embedding
    """
    
    # TODO: 여기에 Embeddings API 호출 코드 작성
    # response = client.embeddings.create(...)
    # return response.data[0].embedding
    
    return []

# 임베딩 테스트
print("\n[테스트 1] 단어 임베딩")
sample_text = "소방시설 설치 기준"
sample_embedding = get_embedding(sample_text)

if sample_embedding:
    print(f"텍스트: {sample_text}")
    print(f"임베딩 차원: {len(sample_embedding)}")
    print(f"벡터 샘플 (처음 5개): {sample_embedding[:5]}")
else:
    print("[아직 구현되지 않았습니다]")

# ============================================================
# 실습 4: ChromaDB에 저장
# ============================================================

print("\n\n" + "="*60)
print("[실습 4] Vector Database 구축 - ChromaDB")
print("="*60)

# ChromaDB 초기화 (이미 구현됨)
print("\n[진행중] ChromaDB 초기화...")
chroma_client = chromadb.Client(Settings(
    persist_directory="./chroma_db",
    anonymized_telemetry=False
))

try:
    chroma_client.delete_collection("fire_law")
except:
    pass

fire_law_collection = chroma_client.create_collection(
    name="fire_law",
    metadata={"description": "소방기본법 10페이지"}
)
print("[완료] 컬렉션 생성 완료")

# TODO: 청크 임베딩 및 저장
print("\n[실습] 청크를 Vector DB에 저장하기")
print("""
TODO: 각 청크를 임베딩하여 ChromaDB에 저장하세요

힌트:
1. for 반복문으로 law_chunks 순회
2. 각 청크의 text를 get_embedding()으로 임베딩 생성
3. fire_law_collection.add()로 저장
   - ids: [chunk['id']]
   - embeddings: [embedding]
   - documents: [chunk['text']]
   - metadatas: [{"page": chunk['page'], "source": "소방기본법"}]

예시 코드:
for i, chunk in enumerate(law_chunks):
    embedding = get_embedding(chunk['text'])
    fire_law_collection.add(
        ids=[chunk['id']],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "소방기본법"}]
    )
""")

if law_chunks and sample_embedding:
    # TODO: 여기에 반복문 코드를 작성하세요
    saved_count = 0  # 실제로 저장한 청크 수로 업데이트하세요
    
    if saved_count > 0:
        print(f"[완료] {saved_count}개 청크 저장 완료!")
    else:
        print("[아직 구현되지 않았습니다]")
else:
    print("[청킹과 임베딩을 먼저 구현해야 합니다]")

# ============================================================
# 실습 5: RAG 검색 및 답변 생성
# ============================================================

print("\n\n" + "="*60)
print("[실습 5] RAG 시스템 - 검색 후 답변 생성")
print("="*60)

def search_fire_law(query: str, n_results: int = 3) -> List[Dict]:
    """
    소방기본법 검색 함수
    
    TODO: 질문을 임베딩하여 유사한 청크 검색하기
    
    힌트:
    1. 질문을 get_embedding()으로 임베딩
    2. fire_law_collection.query()로 검색
       - query_embeddings: [query_embedding]
       - n_results: n_results
    3. 결과를 딕셔너리 리스트로 변환
    """
    
    # TODO: 여기에 검색 코드 작성
    # query_embedding = get_embedding(query)
    # results = fire_law_collection.query(...)
    # return [{"text": ..., "page": ..., "source": ...}, ...]
    
    return []

def answer_with_rag(question: str) -> str:
    """
    RAG 기반 답변 생성
    
    TODO: 검색 결과를 컨텍스트로 활용하여 답변 생성
    
    프로세스:
    1. search_fire_law()로 관련 문서 검색
    2. 검색 결과를 컨텍스트로 구성
    3. LLM에게 컨텍스트와 질문 전달
    4. LLM이 문서 기반 답변 생성
    """
    
    # TODO: 검색 및 답변 생성 구현
    # search_results = search_fire_law(question, n_results=3)
    # context = "\n\n".join([f"[출처: {r['source']}, {r['page']}페이지]\n{r['text']}" for r in search_results])
    # response = client.chat.completions.create(...)
    
    return ""

# ============================================================
# 최종 테스트
# ============================================================

print("\n[최종 테스트] RAG 시스템 실행")
print("="*60)

test_questions = [
    "소방시설 설치 대상은 어떻게 되나요?",
    "소방시설 점검은 언제 해야 하나요?",
    "위반 시 과태료는 얼마인가요?"
]

for q in test_questions:
    print(f"\n[질문] {q}")
    print("-"*50)
    
    # 검색 결과 확인
    results = search_fire_law(q, n_results=2)
    if results:
        print("[검색된 문서]")
        for i, r in enumerate(results, 1):
            print(f"  {i}. {r['source']} {r['page']}페이지")
            print(f"     {r['text'][:150]}...")
        
        # 답변 생성
        print("\n[답변]")
        answer = answer_with_rag(q)
        print(answer if answer else "[아직 구현되지 않았습니다]")
    else:
        print("[검색 기능을 먼저 구현해야 합니다]")
    print("="*60)

# ============================================================
# Lab 2 완료
# ============================================================

print("\n" + "="*60)
print("[완료] Lab 2 완료!")
print("="*60)
print("""
학습 내용:
1. PDF 텍스트 추출 (pdfplumber)
2. 텍스트 청킹 (Chunking)
3. 임베딩 생성 (Embedding)
4. Vector DB 구축 (ChromaDB)
5. RAG 검색 및 답변 생성

핵심 함수:
- search_fire_law(): 소방기본법 검색
- answer_with_rag(): RAG 기반 답변

다음 Lab:
- Lab 3에서는 화재예방 가이드를 추가하여 다중 문서 검색 시스템을 구축합니다.

실습 체크리스트:
[ ] 실습 1: extract_text_from_pdf 함수 완성
[ ] 실습 2: chunk_text 함수 완성
[ ] 실습 3: get_embedding 함수 완성
[ ] 실습 4: ChromaDB 저장 루프 완성
[ ] 실습 5: search_fire_law와 answer_with_rag 함수 완성
""")